In [1]:
import json
import pandas as pd
import numpy as np
import networkx as nx
import jellyfish
import os
import shutil
import subprocess
import requests
from github import Github
from git import Repo
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from zipfile import ZipFile
from filecmp import dircmp
import configparser
from pprint import pprint

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:149: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.16.2
  UserWarning)


In [2]:
config = configparser.ConfigParser()
config.read('authentication.ini')
print(config.sections())
github_oauth = config['github']['api_key']

['github']


In [3]:
project_releases = pd.read_csv('volatile_projects_complete_links_limit50.csv')

In [4]:
project_releases = project_releases[project_releases['project_name'] == 'Okhttp'].head(4)

In [5]:
project_releases.head()

,project_name,project_link,version_name,commit,timestamp
897,Okhttp,https://github.com/square/okhttp,parent-5.0.0-alpha.2,"Commit(sha=""b84627ef4bc43f2096c1b764720f9ff238...",2021-01-30 19:44:19
898,Okhttp,https://github.com/square/okhttp,parent-5.0.0-alpha.1,"Commit(sha=""79d2a8d192d007b8acb2423d3ab6bba3a4...",2021-01-30 18:36:37
899,Okhttp,https://github.com/square/okhttp,parent-4.10.0-RC1,"Commit(sha=""4fd1e8f99833eebdd2e99f3456322aa197...",2020-10-07 03:28:26
900,Okhttp,https://github.com/square/okhttp,parent-4.9.1,"Commit(sha=""63dcd95bfa2345bb3f3d4abc6b6dbf36cf...",2021-01-30 18:09:57


In [87]:
project_refactoring_data = pd.read_csv('Effort_Estimation_Results/Okhttp.csv')
project_refactoring_data.head()

,file,class,type,cbo,wmc,dit,rfc,lcom,totalMethods,staticMethods,...,modifiers,latest_commit,commit_compared_with,num_dependency,num_line_affected,actual_num_of_classes_touched,latest_commit_date,commit_compared_with_date,author,author_email
0,C:\Users\tanji\Desktop\SoftwareRemodularizatio...,mockwebserver3.CustomDispatcherTest,class,7,4,1,13,0,4,0,...,1,b84627ef4bc43f2096c1b764720f9ff2387dca32,3e331c108905a97fa9718b40844ddc1356fc86b5,4.0,67,1,2021-01-30 19:44:19,2020-10-18 12:13:09,Jesse Wilson,jesse@swank.ca
1,C:\Users\tanji\Desktop\SoftwareRemodularizatio...,mockwebserver3.RecordedRequestTest,class,7,5,2,6,0,5,0,...,1,b84627ef4bc43f2096c1b764720f9ff2387dca32,3e331c108905a97fa9718b40844ddc1356fc86b5,9.0,90,2,2021-01-30 19:44:19,2020-10-18 12:13:09,Jesse Wilson,jesse@swank.ca
2,C:\Users\tanji\Desktop\SoftwareRemodularizatio...,mockwebserver3.internal.http2.Http2Server,class,9,33,2,53,32,9,1,...,17,b84627ef4bc43f2096c1b764720f9ff2387dca32,3e331c108905a97fa9718b40844ddc1356fc86b5,86.0,147,3,2021-01-30 19:44:19,2020-10-18 12:13:09,Jesse Wilson,jesse@swank.ca
3,C:\Users\tanji\Desktop\SoftwareRemodularizatio...,mockwebserver3.MockWebServerTest,class,22,54,1,111,0,40,1,...,17,b84627ef4bc43f2096c1b764720f9ff2387dca32,3e331c108905a97fa9718b40844ddc1356fc86b5,321.0,566,4,2021-01-30 19:44:19,2020-10-18 12:13:09,Jesse Wilson,jesse@swank.ca
4,C:\Users\tanji\Desktop\SoftwareRemodularizatio...,mockwebserver3.CustomDispatcherTest,class,7,4,1,13,0,4,0,...,1,b84627ef4bc43f2096c1b764720f9ff2387dca32,3e331c108905a97fa9718b40844ddc1356fc86b5,4.0,67,5,2021-01-30 19:44:19,2020-10-18 12:13:09,Jesse Wilson,jesse@swank.ca


In [88]:
unique_releases_commit = pd.unique(project_refactoring_data['latest_commit'])
unique_commit = pd.unique(project_refactoring_data['commit_compared_with'])

In [89]:
print(unique_commit)
print(unique_releases_commit)

['3e331c108905a97fa9718b40844ddc1356fc86b5'
 'f8065acac28a0b86f3bdbd854ac87af0789416d0'
 '963c76864344d0d55ba7421f474c130a958a9649'
 '59ae0c138d6468aac5d92781b19cddc26e04e862'
 '4677beea96f0afb1e061f119e52e0203d4cd3738'
 '4036fa55b72faaad3a14f806094184ade157917d'
 'ff959f8fd550b8f55a47038ea4235b1b2ed2eaa8'
 '083315a473d8b2fc972a70449272b296683ac38c'
 '5f2732de5a5bc389be2e5bd72441f2a7198b4ca1'
 '334eb0072512155e57a809b2fa18861c0d68ec9f']
['b84627ef4bc43f2096c1b764720f9ff2387dca32'
 '79d2a8d192d007b8acb2423d3ab6bba3a4f999e1'
 '4fd1e8f99833eebdd2e99f3456322aa197f5e652']


In [90]:
unique_commits = list(set(unique_commit).symmetric_difference(set(unique_releases_commit)))

In [91]:
unique_commits

['f8065acac28a0b86f3bdbd854ac87af0789416d0',
 '4677beea96f0afb1e061f119e52e0203d4cd3738',
 '963c76864344d0d55ba7421f474c130a958a9649',
 '79d2a8d192d007b8acb2423d3ab6bba3a4f999e1',
 '4036fa55b72faaad3a14f806094184ade157917d',
 '334eb0072512155e57a809b2fa18861c0d68ec9f',
 '083315a473d8b2fc972a70449272b296683ac38c',
 'ff959f8fd550b8f55a47038ea4235b1b2ed2eaa8',
 'b84627ef4bc43f2096c1b764720f9ff2387dca32',
 '3e331c108905a97fa9718b40844ddc1356fc86b5',
 '59ae0c138d6468aac5d92781b19cddc26e04e862',
 '4fd1e8f99833eebdd2e99f3456322aa197f5e652',
 '5f2732de5a5bc389be2e5bd72441f2a7198b4ca1']

In [10]:
project_link = "https://github.com/square/okhttp"

## To obtain total number of pages of issues

In [35]:
owner = 'square'
repo = "okhttp"
query_url = f"https://api.github.com/repos/square/okhttp/issues?per_page=100"
params = {
    "state": "closed",
}
headers = {'Authorization': f'token {github_oauth}'}
r = requests.get(query_url, headers=headers, params=params)
print(len(r.json()))

counter = 0
project_name = 'Okhttp'

with open(f'raw_issues_github_json/{project_name}/{project_name}_{counter}.json', 'w', encoding='utf-8') as f:
    json.dump(r.json(), f, ensure_ascii=False, indent=4)
    
while 'next' in r.links:
    query_url = r.links['next']['url']
    r = requests.get(query_url, headers=headers)
    counter += 1
    with open(f'raw_issues_github_json/{project_name}/{project_name}_{counter}.json', 'a', encoding='utf-8') as f:
        json.dump(r.json(), f, ensure_ascii=False, indent=4)
    
    
f.close()

100


In [70]:
main_arr = []

root_dir = 'raw_issues_github_json/Okhttp'
data_counter = 0
for filename in os.listdir(root_dir):
    print(filename)

    with open(f'{root_dir}/{filename}', encoding='utf-8') as f:
        data = json.load(f)
        #print(len(data))
        data_counter += len(data)
        
        
        for issues in data:
            
            if 'pull_request' in issues:
                
                query_url = issues['pull_request']['url'] + "/commits"
                #print(query_url)
                headers = {'Authorization': f'token {github_oauth}'}
                r = requests.get(query_url, headers=headers)
                for element in r.json():
                    temp_arr = []
                    #print(element['sha'])
                    temp_arr.append(issues['id'])
                    temp_arr.append(issues['title'])
                    temp_arr.append(issues['body'])
                    temp_arr.append(issues['created_at'])
                    temp_arr.append(issues['updated_at'])
                    temp_arr.append(issues['closed_at'])
                    temp_arr.append(element['sha'])
                    main_arr.append(temp_arr)
                
            

Okhttp_0.json
Okhttp_1.json
Okhttp_10.json
Okhttp_11.json
Okhttp_12.json
Okhttp_13.json
Okhttp_14.json
Okhttp_15.json
Okhttp_16.json
Okhttp_17.json
Okhttp_18.json
Okhttp_19.json
Okhttp_2.json
Okhttp_20.json
Okhttp_21.json
Okhttp_22.json
Okhttp_23.json
Okhttp_24.json
Okhttp_25.json
Okhttp_26.json
Okhttp_27.json
Okhttp_28.json
Okhttp_29.json
Okhttp_3.json
Okhttp_30.json
Okhttp_31.json
Okhttp_32.json
Okhttp_33.json
Okhttp_34.json
Okhttp_35.json
Okhttp_36.json
Okhttp_37.json
Okhttp_38.json
Okhttp_39.json
Okhttp_4.json
Okhttp_40.json
Okhttp_41.json
Okhttp_42.json
Okhttp_43.json
Okhttp_44.json
Okhttp_45.json
Okhttp_46.json
Okhttp_47.json
Okhttp_48.json
Okhttp_49.json
Okhttp_5.json
Okhttp_50.json
Okhttp_51.json
Okhttp_52.json
Okhttp_53.json
Okhttp_54.json
Okhttp_55.json
Okhttp_56.json
Okhttp_57.json
Okhttp_58.json
Okhttp_59.json
Okhttp_6.json
Okhttp_60.json
Okhttp_61.json
Okhttp_62.json
Okhttp_63.json
Okhttp_64.json
Okhttp_65.json
Okhttp_66.json
Okhttp_7.json
Okhttp_8.json
Okhttp_9.json


In [71]:
okhttp_issues_df = pd.DataFrame(main_arr)

In [76]:
okhttp_issues_df.to_csv('raw_issues_github_json/Okhttp/Okhttp_issues_commits.csv',index=False)

In [73]:
okhttp_issues_df.columns = ['id','title','body','created_at','updated_at','closed_at','sha']

In [74]:
okhttp_issues_df.head()

,id,title,body,created_at,updated_at,closed_at,sha
0,1039069805,Upgrade to Okio 3.0.0,None,2021-10-29T01:02:51Z,2021-10-29T07:39:57Z,2021-10-29T07:39:57Z,366e8dbae51b197c94fa975a468beb808ece1314
1,1038961862,Don't document mockwebserver3.internal,None,2021-10-28T21:19:22Z,2021-11-01T13:29:32Z,2021-11-01T13:29:32Z,8bf8213bd18cac58ae644ba71b2e26837c544cc1
2,1030395997,Fix EventSources.processResponse to not throw ...,EventSources.processResponse is intended to pr...,2021-10-19T14:40:26Z,2021-10-29T09:27:22Z,2021-10-22T16:51:49Z,90370cfbfff713cf1ceb732298ba030d10cb53fc
3,1030395997,Fix EventSources.processResponse to not throw ...,EventSources.processResponse is intended to pr...,2021-10-19T14:40:26Z,2021-10-29T09:27:22Z,2021-10-22T16:51:49Z,4cc25f7052410cb29a0d75830f57a20d7a9a9cba
4,1030395997,Fix EventSources.processResponse to not throw ...,EventSources.processResponse is intended to pr...,2021-10-19T14:40:26Z,2021-10-29T09:27:22Z,2021-10-22T16:51:49Z,fb8470ea189e9231f616fd7acc89e5e7950854d8


In [93]:
test = ['366e8dbae51b197c94fa975a468beb808ece1314', '8bf8213bd18cac58ae644ba71b2e26837c544cc1']
#test = ['334eb0072512155e57a809b2fa18861c0d68ec9f']

for element in test:
    if okhttp_issues_df['sha'].str.contains(element).any():
        print(element)

366e8dbae51b197c94fa975a468beb808ece1314
8bf8213bd18cac58ae644ba71b2e26837c544cc1


In [54]:
owner = 'square'
repo = "okhttp"
query_url = "https://api.github.com/repos/square/okhttp/pulls/5733/commits"

headers = {'Authorization': f'token {github_oauth}'}
r = requests.get(query_url, headers=headers)
print(len(r.json()))


2


In [56]:
for element in r.json():
    print(element['sha'])

87495b7b62888720d5681b0d13e60569f3a869c8
bf5222117611b6a59236a695a6239891bde9fa3c


In [58]:
data[2]

{'url': 'https://api.github.com/repos/square/okhttp/issues/5909',
 'repository_url': 'https://api.github.com/repos/square/okhttp',
 'labels_url': 'https://api.github.com/repos/square/okhttp/issues/5909/labels{/name}',
 'comments_url': 'https://api.github.com/repos/square/okhttp/issues/5909/comments',
 'events_url': 'https://api.github.com/repos/square/okhttp/issues/5909/events',
 'html_url': 'https://github.com/square/okhttp/pull/5909',
 'id': 589630310,
 'node_id': 'MDExOlB1bGxSZXF1ZXN0Mzk1MTMzNTIy',
 'number': 5909,
 'title': 'Defer class initialization to avoid spamming on Conscrypt errors (not for landing)',
 'user': {'login': 'yschimke',
  'id': 231923,
  'node_id': 'MDQ6VXNlcjIzMTkyMw==',
  'avatar_url': 'https://avatars.githubusercontent.com/u/231923?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/yschimke',
  'html_url': 'https://github.com/yschimke',
  'followers_url': 'https://api.github.com/users/yschimke/followers',
  'following_url': 'https://api.github.c

In [48]:
for element in data[3]:
    print(element)

url
repository_url
labels_url
comments_url
events_url
html_url
id
node_id
number
title
user
labels
state
locked
assignee
assignees
milestone
comments
created_at
updated_at
closed_at
author_association
active_lock_reason
pull_request
body
reactions
timeline_url
performed_via_github_app


In [25]:
if 'next' in r.links:
    print('lol')
    
if 'prev' in r.links:
    print('lol2')

lol2


In [23]:
r.links

{'prev': {'url': 'https://api.github.com/repositories/5152285/issues?per_page=100&state=closed&page=66',
  'rel': 'prev'},
 'first': {'url': 'https://api.github.com/repositories/5152285/issues?per_page=100&state=closed&page=1',
  'rel': 'first'}}

In [20]:
r.links['next']

{'url': 'https://api.github.com/repositories/5152285/issues?per_page=100&state=closed&page=2',
 'rel': 'next'}

In [21]:
r.links['next']['url']

'https://api.github.com/repositories/5152285/issues?per_page=100&state=closed&page=2'

In [69]:
query_url = f"https://api.github.com/rate_limit"
print(query_url)
#query_url = f"https://api.github.com/repos/{repo}"
headers = {'Authorization': f'token {github_oauth}'}
r = requests.get(query_url, headers=headers)
pprint(r.json())
print(len(r.json()))

https://api.github.com/rate_limit
{'rate': {'limit': 5000, 'remaining': 4931, 'reset': 1636319461, 'used': 69},
 'resources': {'actions_runner_registration': {'limit': 10000,
                                               'remaining': 10000,
                                               'reset': 1636320299,
                                               'used': 0},
               'code_scanning_upload': {'limit': 500,
                                        'remaining': 500,
                                        'reset': 1636320299,
                                        'used': 0},
               'core': {'limit': 5000,
                        'remaining': 4931,
                        'reset': 1636319461,
                        'used': 69},
               'graphql': {'limit': 5000,
                           'remaining': 5000,
                           'reset': 1636320299,
                           'used': 0},
               'integration_manifest': {'limit': 5000,
            